In [4]:
__author__ = 'Vamshi Guduguntla'
__copyright__ = "NA"
__license__ = "NA"
__version__ = "1.0"

notation= """
Notation:
"?" - moved to a worse solution/ random jump
"." - solution that does not change
"+" - better solution using a local change along one dimension
"""

from __future__ import print_function, division
from time import strftime
from pprint import pprint
import math,random,numpy as np



class Osyczka2:
    
    def __init__(self,num_eval = 0.0,prob = 0.5,high = 10**6,low = -10**6):
        self.n = 6
        self.p = prob
        self.upperbound = [10,10,5,5,6,10]
        self.lowerbound = [0,0,1,1,0,0]
        self.steps = 10
        self.evals = num_eval
        self.baseline_high = high
        self.baseline_low = low
        self.threshold = -100
        self.current_state = self.randomstate() 
        self.reset_baseline()
        self.threshold = self.baseline_low
        
        
    def reset_baseline(self):
        high = -10**6
        low = 10**6
        for _ in range(1000):
            dec = self.randomstate()
            en = self.function_value(dec)
            
            if en > high:
                high = en
            if en < low:
                low = en
        
        self.baseline_high = high
        self.baseline_low = low
        
        
    def randomstate(self):
        while True:
            dec = list()
            for low,high in zip(self.lowerbound,self.upperbound):
                dec.append(random.randrange(low,high))
            if self.contraint_ok(dec):
                return dec   
        
    def function_value(self,dec):
        f1 = -(25*(dec[0]-2)**2 + (dec[1]-2)**2 + ((dec[2]-1)**2)*(dec[3]-4)**2 + (dec[4]-1)**2)
        f2 = dec[0]**2 + dec[1]**2 + dec[2]**2 + dec[3]**2 + dec[4]**2 + dec[5]**2
        return f1+f2 
    
    def norm(self,state):
        return (self.function_value(state) - self.baseline_low)/(self.baseline_high - self.baseline_low)

    def contraint_ok(self,dec):
        g1 =  dec[0] + dec[1]-2 >= 0 
        g2 =  6 - dec[0] - dec[1] >= 0
        g3 =  2 - dec[1] + dec[0] >= 0
        g4 =  2 - dec[0] + 3 * dec[1] >= 0
        g5 =  4 - (dec[2] - 3)**3 + dec[5] - 4 >= 0
        g6 =  (dec[4] - 3)**3 + dec[5] -4 >= 0 
        return g1 and g2 and g3 and g4 and g5 and (self.function_value(dec) >= self.threshold)
    
    
    def mutate_in_a_dimension(self,state):
        rand_index = random.randrange(0,5)
        temp = list(state)
        if self.p < random.random():
            temp[rand_index] = random.randrange(self.lowerbound[rand_index],self.upperbound[rand_index])
            if self.contraint_ok(temp):
                return temp, "."
            else:
                return state,"?"
        else:
            temp = self.mutate_to_maximize(temp,rand_index)
            if temp == state:
                return temp,"."
            else:
                return temp,"+"
    
    
    def mutate_to_maximize(self,state,rand_index):
        increment = (self.upperbound[rand_index] - self.lowerbound[rand_index])/self.steps
        temp = list(state)
        best = state
        for _ in range(self.steps):
            temp[rand_index] += increment
            self.evals += 1
            if (self.function_value(temp) < self.function_value(best)) and self.contraint_ok(temp):
                best = list(temp)
        state = best
        return state
    
    
    def maxwalksat(self):
        max_tries = 100
        max_changes = 50
        self.reset_baseline()
        print("Minimizing Osyczka2")
        print("Baseline High :",self.baseline_high)
        print("Baseline Low  :",self.baseline_low)
        print(notation)
        best_state = self.randomstate()
        for _ in range(max_tries):
            state = self.randomstate()
            display = str()
            for i in range(max_changes):
                if self.function_value(state) < self.threshold:
                    return state
                else:
                    new_state, symbol = self.mutate_in_a_dimension(state)
                display += symbol
                if(self.function_value(state)<self.function_value(best_state)):
                    best_state = state
                if(self.function_value(new_state)<self.function_value(best_state)):
                    best_state = new_state
            print(display,round(self.norm(best_state),5))
        return np.around(best_state,3),"f1+f2:",self.function_value(best_state),"Evals: ",self.evals

In [5]:
model = Osyczka2()
model.maxwalksat()

Minimizing Osyczka2
Baseline High : 134
Baseline Low  : -98

Notation:
"?" - moved to a worse solution/ random jump
"." - solution that does not change
"+" - better solution using a local change along one dimension

....??....?....?...?...?.............???.?.?.?.... 0.01724
?......?.++.....+..........+..+..+...?..+.......+. 0.01724
?......................?.?.......?............?... 0.01724
+....?.??...........?.++..+.......+.?+............ 0.01724
........................................?...?.?... 0.01724
??.?.+..?...+.+.?.+.+.......?..?.....?+...?.++...+ 0.01724
....++...+..+?...+...?.??...+.....?.+??+.......... 0.01724
.........+++..+..??........+.+.++.?+..+..+.......? 0.01724
+...........?........?...+.?.?..........?..??..?.. 0.01724
....?.?...?.??..?.....?...................+?...+.? 0.01724
......?.............................?.??..?....... 0.01724
.+.......+.+.............+.?......??.?..?+?...?... 0.01724
?+++.?..+..?...+.??....+..?......+.?.?.?.+...?+... 0.01724
?..........?.??.?

(array([5, 1, 4, 3, 3, 9]), 'f1+f2:', -98, 'Evals: ', 25080.0)